### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [5]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL


In [ ]:
# Create an empty list to store the reviews
reviews_list =[]

# loop through pages 0-19
 # create query with a page number
    # API results show 10 articles at a time
for page in range(20):
    query_url = f"{url}q={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&page={page}&api-key={nyt_api_key}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(reviews)                          

        # Print the page that was just retrieved
        for review in reviews["response"]["docs"][:2]:  
            print(f"Page {page} retrieved:")
            
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results on page found {page}.")
        break


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_df = json_normalize(reviews_list)
nyt_df.head()

In [ ]:
# Extract the title from the "headline.main" column and  
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

def extract_title(headline):
    start = headline.find("\u2018") + 1
    end = headline.find("\u2019 Review")
    if start > 0 and end > start:
        return headline[start:end]
    return headline
    
nyt_df['title'] = nyt_df['headline.main'].apply(extract_title)
print(nyt_df[['headline.main', 'title']].head())


In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df.head(5)

In [ ]:
# Create a list from the "title" column using to_list()
titles = reviews_df['title'].to_list()
# These titles will be used in the query for The Movie Database
titles

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
# Loop through the titles

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
    print("Sleeping for a while to avoid rate limit.")
    time.sleep(1)
    

    # Add 1 to the request counter
request_counter += 1
    
    # Perform a "GET" request for The Movie Database
search_url = f"https://api.themoviedb.org/3/search/movie?query={title}&api_key={tmdb_api_key}"
    

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
try:
        search_response = requests.get(search_url)
        search_results = search_response.json()
        if search_results['results']:
        # Get movie id
        movie_id = search_results['results'][0]['id']

        # Make a request for a the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_details_url)
        movie_details = movie_response.json()

        # Execute "GET" request with url

        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_details.get('genres', [])]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in movie_details.get('spoken_languages',[])]
       
        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_details.get('production_countries', [])]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data = {
                'title': title,
                'original_title': movie_details.get('original_title'),
                'budget': movie_details.get('budget'),
                'original_language': movie_details.get('original_language'),
                'homepage': movie_details.get('homepage'),
                'overview': movie_details.get('overview'),
                'popularity': movie_details.get('popularity'),
                'runtime': movie_details.get('runtime'),
                'revenue': movie_details.get('revenue'),
                'release_date': movie_details.get('release_date'),
                'vote_average': movie_details.get('vote_average'),
                'vote_count': movie_details.get('vote_count'),
                'genres': genres,
                'spoken_languages': spoken_languages,
                'production_countries': production_countries
            }
        
        # Print out the title that was found
        tmdb_movies_list.append(movie_data)
            print(f"{title} found and processed.")
       else:
            print(f"{title} not found.")
    
except Exception as e:
        print(f"An error occurred for {title}: {e}")


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
